<a href="https://colab.research.google.com/github/sandeepjunaghare/llm/blob/main/cohere_ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# practical applications - modern search engines, eCommerce product recomendations, social media content moderations, email spam filtering, customer support conversational agents


In [4]:
! pip install cohere altair -q

In [2]:
import cohere
import numpy as np
import pandas as pd
import altair as alt

co = cohere.Client("EwcsPBLocPue4X4ZwYKNIMiUM1pKgLd43JMl7bZo")


In [13]:
#load the dataset to a dataframe
from google.colab import files
import io

uploaded = files.upload()

for fn in uploaded.keys():
  print('local uploaded file:  "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  df_orig = pd.read_csv(io.BytesIO(uploaded[fn]),names=['intent','query'])



#print('User uploaded file "{fn}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
#df_orig = pd.read_csv('/Users/sandeep/Dropbox/dev/projects/cohere-ai/data-sets/atis/atis_intents_train.csv',names=['intent','query'])


# Take a small sample for illustration purposes
sample_classes = ['atis_airfare', 'atis_airline', 'atis_ground_service']
df = df_orig.sample(frac=0.12, random_state=30)
df = df[df.intent.isin(sample_classes)]
df_orig = df_orig.drop(df.index)
df.reset_index(drop=True,inplace=True)

# Remove unnecessary column
intents = df['intent'] #save for a later need
df.drop(columns=['intent'], inplace=True)
df.head()



Saving atis_intents_train.csv to atis_intents_train (1).csv
local uploaded file:  "atis_intents_train (1).csv" with length 381136 bytes


,query
0,which airlines fly from boston to washington ...
1,show me the airlines that fly between toronto...
2,show me round trip first class tickets from n...
3,i'd like the lowest fare from denver to pitts...
4,show me a list of ground transportation at bo...


In [5]:
# get the text embeddings

In [17]:
# funciton to retrieve text embedding

def get_embeddings(texts, model='embed-english-v3.0', input_type="search_document"):

  output = co.embed(
                  texts=texts,
                  model=model,
                  input_type=input_type)
  return output.embeddings



In [18]:
# Embed the dataset
df['query_embeds'] = get_embeddings(df['query'].tolist())
df.head()



,query,query_embeds
0,which airlines fly from boston to washington ...,"[0.026550293, 0.012084961, -0.00881958, 0.0113..."
1,show me the airlines that fly between toronto...,"[0.013084412, 0.01776123, -0.014343262, -0.003..."
2,show me round trip first class tickets from n...,"[0.02053833, -0.038482666, 0.061523438, 0.0099..."
3,i'd like the lowest fare from denver to pitts...,"[0.0016889572, 0.015411377, -0.029052734, 0.03..."
4,show me a list of ground transportation at bo...,"[0.03793335, -0.008010864, -0.002319336, -0.01..."


In [21]:
# reduce dimensions into PCA
from sklearn.decomposition import PCA

def get_pca(array,n):
  pca = PCA(n_components=n)
  pca_transform = pca.fit_transform(array)

  return pca_transform


In [22]:
# reduce embeds to 10 principal components embeds
embeds = np.array(df['query_embeds'].tolist())
embeds_pc = get_pca(embeds,10)

In [24]:
# reshape the data for visualization puroposes

sample = 9

source = pd.DataFrame(embeds_pc)[:sample]
source = pd.concat([source, df['query']], axis=1)
source = pd.melt(source, id_vars=['query'])

# configure the chart
# Configure the plot
chart = alt.Chart(source).mark_rect().encode(
    x=alt.X('variable:N', title="Embedding"),
    y=alt.Y('query:N', title='',axis=alt.Axis(labelLimit=500)),
    color=alt.Color('value:Q', title="Value", scale=alt.Scale(
                range=["#917EF3", "#000000"]))
)

result = chart.configure(background='#ffffff'
        ).properties(
        width=700,
        height=400,
        title='Embeddings with 10 dimensions'
       ).configure_axis(
      labelFontSize=15,
      titleFontSize=12)

# Show the plot
result

alt.Chart(...)

In [25]:
# Function to generate the 2D plot
def generate_chart(df,xcol,ycol,lbl='on',color='basic',title=''):
  chart = alt.Chart(df).mark_circle(size=500).encode(
    x=
    alt.X(xcol,
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),

    y=
    alt.Y(ycol,
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),

    color= alt.value('#333293') if color == 'basic' else color,
    tooltip=['query']
    )

  if lbl == 'on':
    text = chart.mark_text(align='left', baseline='middle',dx=15, size=13,color='black').encode(text='query', color= alt.value('black'))
  else:
    text = chart.mark_text(align='left', baseline='middle',dx=10).encode()

  result = (chart + text).configure(background="#FDF7F0"
        ).properties(
        width=800,
        height=500,
        title=title
       ).configure_legend(
  orient='bottom', titleFontSize=18,labelFontSize=18)

  return result

In [26]:
df_pc2.head()

NameError: ignored

In [ ]:
# reduce the embeddings to 2D for visualization purposes

df_pc2 = df.

# concate the pca to the dataframe

# generate the charts